# Kostnadsoverslag closed source LLMs

Data om kostnader og benchmarks er hentet fra: https://www.vellum.ai/llm-leaderboard

In [ ]:
import pandas as pd
import numpy as np

from cost_model import make_df_cost, calculate_total_cost
from plot_funcs import plot_multiple_lines

# Import and merge together
kost = pd.read_csv("kostnader.csv")
bench = pd.read_csv("llm_benchmarks.csv")

df = (
    pd.merge(kost, bench, on=["Modell"], how="left", validate=None)
    .set_index("Modell")
    .drop("Total", axis=1)
)

df.head()


In [ ]:
# Clan str signs in numerical data
for column in df.columns:
    if df[column].dtype == object:
        df[column] = df[column].str.replace('K', '').astype(float)

# Set models column til index for operasjon under
df = df.reset_index()
df.columns = df.columns.str.lower()

df

 ## Lag kost / benchmark plot

## Scenario analyse med kjøpte modeller og bruk av API-tilgang

Analysen antar:
- 800 ansatte
- De sender mellom 50 og 350 requests per dag (inkl. automatiserte prosesser som gjør requests) til modellenes API
- 50 ord og 350 ord per spørsmål
- 100 til 600 ord per svar
- 1.33 tokens per ord

Spesielt requests antas å være konservativt da et automatisert program som gjør klassifisering eller datainnhenting kan medføre mange fler requests.
Spørsmålslengden er også svært konservativ da konteksten fra selv enkle RAG modeller gjør at man potensielt får mye lengre spørsmål enn 350 ord (spesielt hvis man prøver å oppsummere dokumentasjon). Tokens per ord kan også variere med hvilken embedder-modell som brukes og hvilket språk man sender spørsmål på.

In [ ]:

# Parametre for modellen
employees = 800
calls = np.arange(50, 350, 50)
avg_words_per_q = np.arange(50, 350, 50)
avg_words_per_a = np.arange (100, 700, 100)
tokens_per_word = 1.33

# Dra ut subset columns med modellkostnader til bruk i modell
df_llms = df[['modell', 'context window', 'input cost / 1m tokens','output cost / 1m tokens']].copy()
df_llms.loc[:, 'context window'] = df_llms.loc[:, 'context window'] * 1000

In [ ]:
# Lager en stor df med enhetskostnader knyttet til parametre i modellen
df_cost = make_df_cost(employees, calls.tolist(), avg_words_per_q.tolist(), avg_words_per_a.tolist(), tokens_per_word)
    

In [ ]:
# Vis dataframe
df_cost

In [ ]:
cost_sent = 'input cost / 1m tokens'
cost_rec = 'output cost / 1m tokens'
models = ['Gemini 1.5 Pro', 'GPT-4', 'Mistral Large', 'Claude 3 Opus']

# Kalkuler totale kostnader
total_cost = calculate_total_cost(df_cost, df_llms, models, cost_sent, cost_rec)


In [ ]:
# Make totals columns for tokens
total_cost['m_tokens_total'] = total_cost.loc[:, 'm_tokens_sent'] + total_cost.loc[:, 'm_tokens_received']

In [ ]:
# Check largest combinations
total_cost.loc[total_cost['m_tokens_total'] >= 90]

## Plotting av resultater

In [ ]:
fig1 = plot_multiple_lines(total_cost, 
               'm_tokens_total',
               total_cost.columns[total_cost.columns.str.contains('Total cost')],
               title='Token amount vs expenditure',
               xaxis_title='Million tokens (sent + received)',
               yaxis_title='Daily expenditure (USD)')

fig1.savefig('plots/tokens_vs_cost.png')
            

In [ ]:
fig2 = plot_multiple_lines(total_cost.sort_values('m_tokens_received'), 
               'm_tokens_received', 
               total_cost.columns[total_cost.columns.str.contains('Answer cost')],
               title='Answers vs expenditure',
               xaxis_title='Million tokens received',
               yaxis_title='Daily expenditure (USD)')


fig2.savefig('plots/received_tokens_vs_expenditure.png')

In [ ]:
fig3 = plot_multiple_lines(total_cost.sort_values('m_tokens_sent'), 
               'm_tokens_sent', 
               total_cost.columns[total_cost.columns.str.contains('Question cost')],
               title='Questions vs expenditure',
               xaxis_title='Million tokens sent',
               yaxis_title='Daily expenditure (USD)')


fig3.savefig('plots/tokens_sent_vs_expenditure.png')

## Ekport av relevante datasett

In [ ]:
# Kolonner til eksporterte datasett
cols_avg_word_per_a = [
    "avg_words_per_a",
    "avg_words_per_q",
    "calls",
    "m_tokens_total",
    "Total cost Gemini 1.5 Pro (USD)",
    "Total cost GPT-4 (USD)",
]

cols_n_calls = [
    "avg_words_per_a",
    "avg_words_per_q",
    "calls",
    "m_tokens_total",
    "Total cost Gemini 1.5 Pro (USD)",
    "Total cost GPT-4 (USD)",
]

# Lag filtreringsmasker
words_per_a_mask = ((total_cost["avg_words_per_a"] >= 100) | (total_cost["avg_words_per_a"] <= 200))
n_calls_mask = total_cost["calls"] == 50

# Sensitivitet mtp mengde requests og antall ord per svar, send til datafil for bruk i ppt
total_cost.loc[words_per_a_mask, cols_avg_word_per_a].to_excel(
    "eksporterte_data/kostnad_ord_per_svar.xlsx"
)
total_cost.loc[n_calls_mask, cols_n_calls].to_excel(
    "eksporterte_data/kostnad_calls.xlsx"
)

# Skriv heile shiten til disk, liker ikke excel men la gå
total_cost.to_excel("eksporterte_data/totale_kostnadsberegninger.xlsx")